In [1]:
import boto3
from sagemaker import get_execution_role
import json
import pickle
import flair
import sagemaker
import torch
import tarfile

role = get_execution_role()
print(torch.cuda.is_available()) # we have gpus available (ml.p2.xlarge)
print(torch.cuda.device_count()) # we have 1 GPU on this machine

True
1


In [2]:
# Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
sess = sagemaker.Session()

# uses a default bucket created by sagemaker
bucket = sess.default_bucket()

# Region of our account
region = "us-east-1"

# We will use these values when using batch transform
prefix_input = 'flair-input'
prefix_output = 'flair-ouput'

### Lets download the pre-trained sequence model

- We also need to save this model and then tar.gz it in order to make it compatible with sagemaker for inference
    - This applies for both realtime endpoints and for batch transform.

In [ ]:
from flair.models import SequenceTagger

# Will download the model with pre-trained weights
model = SequenceTagger.load('ner')

# Saving the model to disk
torch.save(model, "flair_model.pth")

# Utility function to wrap model inside a tar.gz
def tar_gz_model(model_filename):
    with tarfile.open('model.tar.gz', 'w:gz') as f:
        f.add(f'./{model_filename}')
        

#creating tar.gz file
tar_gz_model('flair_model.pth')

### Let's now store this model artifact on S3 where sagemaker can find it

In [ ]:
!aws s3 cp model.tar.gz s3://pytorch-flair-test/model.tar.gz

### Let's now do some local mode inferencing

- Local mode is a great way to iteratively experiment with your model as well as any associated scripts for inferencing.
- Local mode allows you to spin up a container on your notebook instance, that is identical to a dedicated endpoint, from an environment perspective.

In [3]:
# We want to create a PyTorchModel from our model.tar.gz
from sagemaker.pytorch.model import PyTorchModel

In [4]:
# This is the inference script we will provide our predictor - take a moment to read through it.
!pygmentize -l python source_dir/inference.py

__authors__ = ['Ibrahim Gabr', 'Vadim Dabravolski']

import json
import pickle
import logging
import sys
import os
import torch

from flair.data import Sentence

JSON_CONTENT_TYPE = 'application/json'
CSV_CONTENT_TYPE = 'text/csv'
PICKLE_CONTENT_TYPE = 'pickle'


# Ensure logging to /logs/mms_logs.log in the container.
# Logging everything from INFO level and above
# Stream output to stdout -> will be visible in docker container stdout in local mode and in cloud watch when using dedicated endpoints.
logging.basicConfig(stream=sys.stdout, format="%(message)s", level=logging.INFO)

## NOTE:
"""
The methods input_fn and output_fn are OPTIONAL.
If obmitted SageMaker will assume:

the input and output objects are of type NPY format with Content-Type application/x-npy.
"""

def model_fn(model_dir):
    """
    A function to load up your model into memory.

    The return of this function is passed to predict_fn at the time of inference.

    This function is only run once at the start if the

### Let's get some Realtime predictions from our local container using SageMaker SDK

Recall, the default behaviour fo the predictor is accept (de)serializations in NPY format. We will have to alter this for our use case.

This will not be required when using the boto3 invoke_endpoint API.

In [5]:
from sagemaker.predictor import json_serializer, RealTimePredictor

In [6]:
# Have to use local session here
local_sess = sagemaker.LocalSession()

# We are going to serialize the inputs as JSON (recall default is NPY)
# We do not need to perform any deserialization of the results as we will be returning the object as a pickle
# execute the logic for 'application/json' in the input function
# execute the logic for 'pickle' in the output function

class FLAIRPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=local_sess, serializer=json_serializer, 
                         deserializer=None, content_type='application/json', accept='pickle')

In [7]:
# Model_data = where is the model artifact? Can also point to model on disk
# Role = AWS IAM role associated with the notebook - permissions
# Entrypoint = this is the inferencing script used to serve and recieve requests - we will show this
#framework version = what version of pytorch would you like to use
#Source dir = the folder or absolute/relative path containing the inference scripts AND any requirements.txt for the container
pytorch_model = PyTorchModel(model_data='s3://pytorch-flair-test/model.tar.gz',
                             role=role,
                             entry_point='inference.py',
                             framework_version='1.4.0',
                             source_dir='source_dir',
                             predictor_cls=FLAIRPredictor)

In [8]:
# initial_instance_count = 1 (1 container locally in this case)
# instance type = 'local_gpu' - we want to use the attached gpu and not the CPU
pytorch_model_local = pytorch_model.deploy(initial_instance_count=1, instance_type='local_gpu')

Attaching to tmpubwo1hvh_algo-1-lr1d3_1
algo-1-lr1d3_1  | Collecting flair
algo-1-lr1d3_1  |   Downloading flair-0.5-py3-none-any.whl (334 kB)
     |████████████████████████████████| 334 kB 22.2 MB/s 
algo-1-lr1d3_1  | Requirement already satisfied: torch>=1.1.0 in /opt/conda/lib/python3.6/site-packages (from flair->-r /opt/ml/model/code/requirements.txt (line 1)) (1.4.0)
algo-1-lr1d3_1  | Collecting hyperopt>=0.1.1
algo-1-lr1d3_1  |   Downloading hyperopt-0.2.4-py2.py3-none-any.whl (964 kB)
     |████████████████████████████████| 964 kB 38.9 MB/s 


algo-1-lr1d3_1  | Collecting regex
algo-1-lr1d3_1  |   Downloading regex-2020.5.14-cp36-cp36m-manylinux2010_x86_64.whl (675 kB)
     |████████████████████████████████| 675 kB 44.3 MB/s 
algo-1-lr1d3_1  | Collecting tabulate
algo-1-lr1d3_1  |   Downloading tabulate-0.8.7-py3-none-any.whl (24 kB)
algo-1-lr1d3_1  | Collecting mpld3==0.3
algo-1-lr1d3_1  |   Downloading mpld3-0.3.tar.gz (788 kB)
     |████████████████████████████████| 788 kB 18.3 MB/s 


algo-1-lr1d3_1  | Collecting deprecated>=1.2.4
algo-1-lr1d3_1  |   Downloading Deprecated-1.2.10-py2.py3-none-any.whl (8.7 kB)
algo-1-lr1d3_1  | Collecting bpemb>=0.2.9
algo-1-lr1d3_1  |   Downloading bpemb-0.3.0-py3-none-any.whl (19 kB)
algo-1-lr1d3_1  | Requirement already satisfied: python-dateutil>=2.6.1 in /opt/conda/lib/python3.6/site-packages (from flair->-r /opt/ml/model/code/requirements.txt (line 1)) (2.8.1)
algo-1-lr1d3_1  | Collecting pytest>=5.3.2
algo-1-lr1d3_1  |   Downloading pytest-5.4.3-py3-none-any.whl (248 kB)
     |████████████████████████████████| 248 kB 47.2 MB/s eta 0:00:01
algo-1-lr1d3_1  | Collecting matplotlib>=2.2.3
algo-1-lr1d3_1  |   Downloading matplotlib-3.2.1-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
     |████████████████████████████████| 12.4 MB 36.0 MB/s eta 0:00:01
algo-1-lr1d3_1  | Collecting sqlitedict>=1.6.0
algo-1-lr1d3_1  |   Downloading sqlitedict-1.6.0.tar.gz (29 kB)
algo-1-lr1d3_1  | Collecting transformers>=2.10.0
algo-1-lr1d3_1  |   Downl

     |████████████████████████████████| 1.4 MB 28.9 MB/s 
algo-1-lr1d3_1  | Requirement already satisfied: boto3 in /opt/conda/lib/python3.6/site-packages (from smart-open>=1.8.1->gensim>=3.4.0->flair->-r /opt/ml/model/code/requirements.txt (line 1)) (1.12.35)
algo-1-lr1d3_1  | Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /opt/conda/lib/python3.6/site-packages (from boto3->smart-open>=1.8.1->gensim>=3.4.0->flair->-r /opt/ml/model/code/requirements.txt (line 1)) (0.9.5)
algo-1-lr1d3_1  | Requirement already satisfied: s3transfer<0.4.0,>=0.3.0 in /opt/conda/lib/python3.6/site-packages (from boto3->smart-open>=1.8.1->gensim>=3.4.0->flair->-r /opt/ml/model/code/requirements.txt (line 1)) (0.3.3)
algo-1-lr1d3_1  | Requirement already satisfied: botocore<1.16.0,>=1.15.35 in /opt/conda/lib/python3.6/site-packages (from boto3->smart-open>=1.8.1->gensim>=3.4.0->flair->-r /opt/ml/model/code/requirements.txt (line 1)) (1.15.35)
algo-1-lr1d3_1  | Requirement already satisfied: docutils

algo-1-lr1d3_1  |   Building wheel for mpld3 (setup.py) ... done
algo-1-lr1d3_1  |   Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116678 sha256=ce738d41b1b7a14d20b516f9707d5d4258739a5f8131ac2c8c73fdbf3e41ba21
algo-1-lr1d3_1  |   Stored in directory: /root/.cache/pip/wheels/c0/74/c9/ac92f0c4c9eb137d440e86c2822aba0b96b63e608dd5737164
algo-1-lr1d3_1  |   Building wheel for sqlitedict (setup.py) ... done
algo-1-lr1d3_1  |   Created wheel for sqlitedict: filename=sqlitedict-1.6.0-py3-none-any.whl size=14688 sha256=4b9f0c7eeb7da61b08a8b832a0991463893b5d15937b03345a3878cee196b608
algo-1-lr1d3_1  |   Stored in directory: /root/.cache/pip/wheels/7d/44/14/5dc41bad7fa0e87462127d2beba1eaae0c180c98f1024a31db
algo-1-lr1d3_1  |   Building wheel for langdetect (setup.py) ... done
algo-1-lr1d3_1  |   Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993191 sha256=df918ea93375b74e5d56a1d12e01f8a6953bdff226192a41139991043a0ad47d
algo-1-lr1d3_1  |   Stored 

In [9]:
sentence_to_predict  = "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."
sentence_to_predict

'New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States.'

In [10]:
# Taking our string -> serializing it as a JSON payload -> loaded as a string in input_function -> returns pickle object
pickle.loads(pytorch_model_local.predict(sentence_to_predict))

algo-1-lr1d3_1  | 2020-06-03 18:03:43,120 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Input serialization succesfully completed.
algo-1-lr1d3_1  | 2020-06-03 18:03:43,457 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Model prediction: [Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]
algo-1-lr1d3_1  | 2020-06-03 18:03:43,458 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Output serilaization sucessfully completed. 
algo-1-lr1d3_1  | 2020-06-03 18:03:43,459 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 340
algo-1-lr1d3_1  | 2020-06-03 18:03:43,460 [INFO ] W-9000-model ACCESS_L

[Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]

In [11]:
from sagemaker.predictor import json_deserializer, csv_deserializer

In [12]:
# lets get our output as 'application/json'
pytorch_model_local.accept = 'application/json'

In [13]:
# lets deserialize the response (i.e. json.loads() automatically as the output_fn serialized the response into JSON (look at inference.py!))
pytorch_model_local.deserializer = json_deserializer

In [14]:
pytorch_model_local.predict(sentence_to_predict)

algo-1-lr1d3_1  | 2020-06-03 18:03:52,555 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Input serialization succesfully completed.
algo-1-lr1d3_1  | 2020-06-03 18:03:52,706 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Model prediction: [Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]
algo-1-lr1d3_1  | 2020-06-03 18:03:52,706 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 152


'[Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]'

algo-1-lr1d3_1  | 2020-06-03 18:03:52,706 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Output serilaization sucessfully completed. 
algo-1-lr1d3_1  | 2020-06-03 18:03:52,706 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:36172 "POST /invocations HTTP/1.1" 200 153


In [15]:
# Lets get our output as 'text/csv'
pytorch_model_local.accept = 'text/csv'

In [16]:
# There is nothing to deserialize in the response - as its already raw text.
pytorch_model_local.deserializer = None

In [17]:
# comes back as a byte stream of text - so we decode. Notice how this equivalent to the above!
pytorch_model_local.predict(sentence_to_predict).decode('utf-8')

algo-1-lr1d3_1  | 2020-06-03 18:03:57,515 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Input serialization succesfully completed.
algo-1-lr1d3_1  | 2020-06-03 18:03:57,657 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Model prediction: [Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]
algo-1-lr1d3_1  | 2020-06-03 18:03:57,658 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Output serilaization sucessfully completed. 


'[Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]'

algo-1-lr1d3_1  | 2020-06-03 18:03:57,658 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 144
algo-1-lr1d3_1  | 2020-06-03 18:03:57,658 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:36172 "POST /invocations HTTP/1.1" 200 145


In [18]:
# What if we deserialized as csv/text?
pytorch_model_local.deserializer = csv_deserializer

In [19]:
# we get the result back as if we read the result from a CSV file in python
pytorch_model_local.predict(sentence_to_predict)

algo-1-lr1d3_1  | 2020-06-03 18:03:59,395 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Input serialization succesfully completed.
algo-1-lr1d3_1  | 2020-06-03 18:03:59,525 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Model prediction: [Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]]
algo-1-lr1d3_1  | 2020-06-03 18:03:59,525 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Output serilaization sucessfully completed. 


[['[Sentence: "New York City (NYC)',
  ' often called The City or simply New York (NY)',
  ' is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC)',
  ' often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY)',
  ' is the most populous city in the United <B-LOC> States. <E-LOC>"]]']]

algo-1-lr1d3_1  | 2020-06-03 18:03:59,525 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 130
algo-1-lr1d3_1  | 2020-06-03 18:03:59,526 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:36172 "POST /invocations HTTP/1.1" 200 132


In [20]:
#terminate local container - everything is working as expected!
pytorch_model_local.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


## Lets now deploy to an actual endpoint


In [181]:
# Do not need to provide image, as it will infer the latest image associated with pytorch
# Can also provide custom image if you like.

# Note how we do NOT use localsession here

remote_model = PyTorchModel(model_data='s3://pytorch-flair-test/model.tar.gz',
                   role=role,
                   sagemaker_session=sess,
                   entry_point='inference.py',
                   name='flair-sequence-tagger',
                   framework_version='1.4.0',
                   source_dir='source_dir')

In [182]:
remote_predictor = remote_model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge',
                                       endpoint_name='flair-endpoint')

-------------------!

In [183]:
# We will now use the boto3 API to obtain inferences from our endpoint
# Note that the boto3 endpoint, simply submits the raw payload as is! No serilization or deserialization done
# Except for logic associated in our inference.py

sentence_to_predict  = "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."

client = boto3.client('sagemaker-runtime')
content_type = 'application/json'
accept_type = "pickle" 
payload = json.dumps(sentence_to_predict)
endpoint_name = "flair-endpoint"

resp = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType=content_type,
    Accept = accept_type
)

result = pickle.loads(resp['Body'].read())[0]
print(type(result))
print()
print(result)

<class 'flair.data.Sentence'>

Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States."   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States. <E-LOC>"]


### Lets take a look at batch transform!

In [28]:
input_data_path = f's3://{bucket}/{prefix_input}/sample.csv'
output_data_path = f's3://{bucket}/{prefix_output}/'

In [34]:
!aws s3 cp sample.csv $input_data_path

upload: ./sample.csv to s3://sagemaker-us-east-1-544194174732/flair-input/sample.csv


In [174]:
transform_job = sagemaker.transformer.Transformer(
    model_name = "flair-sequence-tagger",
    instance_count = 1,
    instance_type = 'ml.p2.xlarge',
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    base_transform_job_name='flair-transform-batch-transform-1',
    sagemaker_session=sess,
    accept = "text/csv") # note how we are changing the return type

transform_job.transform(data = input_data_path, 
                        content_type = "text/csv",# note how the input type has changed 
                        split_type = 'Line') # one record per line

In [175]:
sm = boto3.client('sagemaker')

In [177]:
import time
while(True):
    response = sm.describe_transform_job(TransformJobName='flair-transform-batch-transform-1-2020-06-03-19-47-29-557')
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Transform job is still in status: " + status)    
    time.sleep(30) 

Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job ended with status: Completed


In [178]:
# our output file
! aws s3 ls $output_data_path

2020-06-03 19:55:23       1036 sample.csv.out


In [179]:
! aws s3 cp "s3://sagemaker-us-east-1-544194174732/flair-ouput/sample.csv.out" results.csv

download: s3://sagemaker-us-east-1-544194174732/flair-ouput/sample.csv.out to ./results.csv


In [180]:
! cat results.csv

[Sentence: "New York City (NYC), often called The City or simply New York (NY), is the most populous city in the United States.
"   [− Tokens: 22  − Token-Labels: "New <B-LOC> York <I-LOC> City <E-LOC> (NYC), often called The <B-LOC> City <E-LOC> or simply New <B-LOC> York <E-LOC> (NY), is the most populous city in the United <B-LOC> States.
 <E-LOC>"]]
[Sentence: "Paris is the capital and most populous city of France, with a population of 2,148,271 residents (official estimate, 1 January 2020) in an area of 105 square kilometres (41 square miles).
"   [− Tokens: 31  − Token-Labels: "Paris <S-LOC> is the capital and most populous city of France, <S-LOC> with a population of 2,148,271 residents (official estimate, 1 January 2020) in an area of 105 square kilometres (41 square miles).
"]]
[Sentence: "Berlin is the capital and largest city of Germany by both area and population.
"   [− Tokens: 14  − Token-Labels: "Berlin <S-LOC> is the capital and largest city of Germany <S-LOC> by both a